In [ ]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed
import nfl_data_py as nfl
import ssl
ssl._create_default_https_context = ssl._create_stdlib_context
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
import warnings
warnings.filterwarnings('ignore')

# nfl-data-py

In [ ]:
# 1999-2021 available for analysis. Some properties are not available in earlier years
seasons = [*range(2022, 2024, 1)]
draft_years = [*range(2000, 2024, 1)]

print(seasons)
print(draft_years)

#### Data dictionary

https://nflreadr.nflverse.com/articles/dictionary_pbp.html

#### Import play-by-play data

In [ ]:
play_data = nfl.import_pbp_data(years=seasons, downcast=True, cache=False) # , alt_path=None)

# Clean dataframe using nfl-data-api cleaning function
play_df = nfl.clean_nfl_data(play_data)

play_df.shape

In [ ]:
# Modify team names to match other data sources
play_df['home_team'] = np.where(play_df['home_team'] == 'OAK', 'LV', play_df['home_team'])
play_df['away_team'] = np.where(play_df['away_team'] == 'OAK', 'LV', play_df['away_team'])
play_df['posteam'] = np.where(play_df['posteam'] == 'OAK', 'LV', play_df['posteam'])
play_df['defteam'] = np.where(play_df['defteam'] == 'OAK', 'LV', play_df['defteam'])

play_df['side_of_field'] = np.where(play_df['side_of_field'] == 'OAK', 'LV', play_df['side_of_field'])

play_df['game_id'] = play_df['game_id'].str.replace('OAK', 'LV', case=True)

# Create teams_date_id
play_df['year'] = pd.DatetimeIndex(play_df['game_date']).year
play_df['month'] = pd.DatetimeIndex(play_df['game_date']).month
play_df['day'] = pd.DatetimeIndex(play_df['game_date']).day

play_df['game_alt_id'] = play_df['home_team'] + '_' + play_df['away_team'] + '_' +  play_df['year'].astype(str) + '_' + play_df['month'].astype(str).str.zfill(2) + '_' + play_df['day'].astype(str).str.zfill(2)
    
play_df.drop(['year','month','day'], axis=1, inplace=True)

play_df.sample(1)

In [ ]:
play_df.home_team.nunique()

In [ ]:
# Print columns with missing values
missing = play_df.isnull().sum()
missing

#### Import Schedule data

In [ ]:
game_data = nfl.import_schedules(years=seasons)

# Clean dataframe using nfl-data-api cleaning function
game_df = nfl.clean_nfl_data(game_data)

# # Sort dataframe
game_df = game_df.sort_values(by=['season','week','gameday'], ascending=True)

game_df.shape

In [ ]:
game_df.head(2)

In [ ]:
# Modify team codes to match schedule
game_df['home_team'] = np.where(game_df['home_team'] == 'OAK', 'LV', game_df['home_team'])
game_df['away_team'] = np.where(game_df['away_team'] == 'OAK', 'LV', game_df['away_team'])
game_df['game_id'] = game_df['game_id'].str.replace('OAK', 'LV', case=True)

In [ ]:
game_df.home_team.nunique()

In [ ]:
game_df.away_team.nunique()

In [ ]:
# Create teams_date_id
game_df['year'] = pd.DatetimeIndex(game_df['gameday']).year
game_df['month'] = pd.DatetimeIndex(game_df['gameday']).month
game_df['day'] = pd.DatetimeIndex(game_df['gameday']).day

game_df['game_alt_id'] = game_df['home_team'] + '_' + game_df['away_team'] + '_' + game_df['year'].astype(str) + '_' + game_df['month'].astype(str).str.zfill(2) + '_' + game_df['day'].astype(str).str.zfill(2)
    
game_df.drop(['year','month','day'], axis=1, inplace=True)

# Create game_id lookup table (joined with QBR table)
game_df = game_df[['game_id','game_alt_id','gameday','weekday','gametime','under_odds','over_odds','away_rest','home_rest']]

game_df.sample(2)

#### Import QBR data

In [ ]:
qbr_data = nfl.import_qbr(years=seasons, level='nfl', frequency='weekly')

# Clean dataframe using nfl-data-api cleaning function
qbr_df = nfl.clean_nfl_data(qbr_data)

qbr_df.shape

In [ ]:
qbr_df.sample(2)

In [ ]:
# Rename columns
qbr_df = qbr_df.rename(columns={'team_abb':'posteam', 'opp_abb':'defteam'})

# Drop column
qbr_df.drop(['game_id'], axis=1, inplace=True)

# Rename columns
qbr_df = qbr_df.rename(columns={'qb_plays': 'plays'})

# Create sequential week column
qbr_df['week'] = np.where((qbr_df['season'] < 2021) & (qbr_df['season_type'] == 'Playoffs'), (17 + qbr_df['game_week']),
                 np.where((qbr_df['season'] >= 2021) & (qbr_df['season_type'] == 'Playoffs'), (18 + qbr_df['game_week']),
                 qbr_df['game_week']))

# Modify team codes to match schedule
qbr_df['posteam'] = np.where(qbr_df['posteam'] == 'OAK', 'LV', qbr_df['posteam'])
qbr_df['defteam'] = np.where(qbr_df['defteam'] == 'OAK', 'LV', qbr_df['defteam'])
qbr_df['posteam'] = np.where(qbr_df['posteam'] == 'LAR', 'LA', qbr_df['posteam'])
qbr_df['defteam'] = np.where(qbr_df['defteam'] == 'LAR', 'LA', qbr_df['defteam'])
qbr_df['posteam'] = np.where(qbr_df['posteam'] == 'WSH', 'WAS', qbr_df['posteam'])
qbr_df['defteam'] = np.where(qbr_df['defteam'] == 'WSH', 'WAS', qbr_df['defteam'])

# Impute incorrect values
qbr_df['week'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'LV') & (qbr_df['defteam'] == 'BAL') & (qbr_df['week'] == 1), 12, qbr_df['week'])
qbr_df['week'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'BAL') & (qbr_df['defteam'] == 'LV') & (qbr_df['week'] == 1), 12, qbr_df['week'])
qbr_df['defteam'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'LV') & (qbr_df['defteam'] == 'CHI') & (qbr_df['week'] == 9), 'SF', qbr_df['defteam'])
qbr_df['defteam'] = np.where((qbr_df['season'] == 2018) & (qbr_df['posteam'] == 'CHI') & (qbr_df['defteam'] == 'LV') & (qbr_df['week'] == 9), 'SF', qbr_df['defteam'])

# Create both combinations of game_id
qbr_df['game_id_1'] = qbr_df['season'].astype(str) + '_' + qbr_df['week'].astype(str).str.zfill(2) + '_' + qbr_df['posteam'] + '_' + qbr_df['defteam']
qbr_df['game_id_2'] = qbr_df['season'].astype(str) + '_' + qbr_df['week'].astype(str).str.zfill(2) + '_' + qbr_df['defteam'] + '_' + qbr_df['posteam']

# Reduce dataframe dimensions
qbr_df = qbr_df[['game_id_1','game_id_2','season','week','posteam','rank','pts_added','plays','qbr_raw','qbr_total','exp_sack','sack','pass','run']]

# Merge with schedule dataframe to get game_id
qbr_df = pd.merge(qbr_df, game_df, how='left', left_on=['game_id_1'], right_on = ['game_id'])
qbr_df = qbr_df.rename(columns={'game_id':'game_id1',
                                'game_alt_id':'game_alt_id1',
                                'gameday':'gameday1',
                                'weekday':'weekday1',
                                'gametime':'gametime1',
                                'under_odds':'under_odds1',
                                'over_odds':'over_odds1',
                                'away_rest':'away_rest1',
                                'home_rest':'home_rest1',
                                })

qbr_df = pd.merge(qbr_df, game_df, how='left', left_on=['game_id_2'], right_on = ['game_id'])
qbr_df = qbr_df.rename(columns={'game_id':'game_id2',
                                'game_alt_id':'game_alt_id2',
                                'gameday':'gameday2',
                                'weekday':'weekday2',
                                'gametime':'gametime2',
                                'under_odds':'under_odds2',
                                'over_odds':'over_odds2',
                                'away_rest':'away_rest2',
                                'home_rest':'home_rest2',
                                })

# Combine columns
qbr_df['game_id'] = np.where(qbr_df['game_id1'].isnull(), qbr_df['game_id2'], qbr_df['game_id1'])
qbr_df['game_alt_id'] = np.where(qbr_df['game_alt_id1'].isnull(), qbr_df['game_alt_id2'], qbr_df['game_alt_id1'])
qbr_df['gameday'] = np.where(qbr_df['gameday1'].isnull(), qbr_df['gameday2'], qbr_df['gameday1'])
qbr_df['weekday'] = np.where(qbr_df['weekday1'].isnull(), qbr_df['weekday2'], qbr_df['weekday1'])

qbr_df['gametime'] = np.where(qbr_df['gametime1'].isnull(), qbr_df['gametime2'], qbr_df['gametime1'])
qbr_df['under_odds'] = np.where(qbr_df['under_odds1'].isnull(), qbr_df['under_odds2'], qbr_df['under_odds1'])
qbr_df['over_odds'] = np.where(qbr_df['over_odds1'].isnull(), qbr_df['over_odds2'], qbr_df['over_odds1'])
qbr_df['away_rest'] = np.where(qbr_df['away_rest1'].isnull(), qbr_df['away_rest2'], qbr_df['away_rest1'])
qbr_df['home_rest'] = np.where(qbr_df['home_rest1'].isnull(), qbr_df['home_rest2'], qbr_df['home_rest1'])

qbr_df['game_id'].fillna('NULL', inplace = True)

qbr_df.drop(['game_id_1','game_id_2','game_id1','game_id2','game_alt_id1','game_alt_id2','gameday1','gameday2','weekday1','weekday2','gametime1','gametime2','under_odds1','under_odds2','over_odds1','over_odds2','away_rest1','away_rest2','home_rest1','home_rest2'], axis=1, inplace=True)

qbr_df.shape

In [ ]:
# Sort dataframe and drop quarterback with fewer passes during week
qbr_df = qbr_df.sort_values(by=['season','posteam','week','plays'], ascending=True)

qbr_df = qbr_df.drop_duplicates(['season','posteam','week'], keep='last')

qbr_df.shape

In [ ]:
# Create rolling average of quarterback statistics
qbr_df['qb_rank'] = qbr_df.groupby(['season',
                                    'posteam'])['rank'].transform(lambda x: x.rolling(window=3,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())

qbr_df['qb_pts'] = qbr_df.groupby(['season',
                                    'posteam'])['pts_added'].transform(lambda x: x.rolling(window=3,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())

qbr_df['qb_plays'] = qbr_df.groupby(['season',
                                    'posteam'])['plays'].transform(lambda x: x.rolling(window=3,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
qbr_df['qb_qbr_raw'] = qbr_df.groupby(['season',
                                    'posteam'])['qbr_raw'].transform(lambda x: x.rolling(window=3,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
qbr_df['qb_qbr'] = qbr_df.groupby(['season',
                                    'posteam'])['qbr_total'].transform(lambda x: x.rolling(window=3,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
# qbr_df['qb_sack'] = qbr_df.groupby(['season',
#                                     'posteam'])['sack'].transform(lambda x: x.rolling(window=3,
#                                                                                       min_periods=1,
#                                                                                       closed='left',
#                                                                                       center=False).mean())
# qbr_df['qb_exp_sack'] = qbr_df.groupby(['season',
#                                     'posteam'])['exp_sack'].transform(lambda x: x.rolling(window=3,
#                                                                                       min_periods=1,
#                                                                                       closed='left',
#                                                                                       center=False).mean())
qbr_df['qb_pass'] = qbr_df.groupby(['season',
                                    'posteam'])['pass'].transform(lambda x: x.rolling(window=3,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())
qbr_df['qb_run'] = qbr_df.groupby(['season',
                                    'posteam'])['run'].transform(lambda x: x.rolling(window=3,
                                                                                      min_periods=1,
                                                                                      closed='left',
                                                                                      center=False).mean())

qbr_df.drop(['rank','pts_added','plays','qbr_raw','qbr_total','exp_sack','sack','pass','run'],  axis=1, inplace=True)

In [ ]:
# Impute missing data with median values
qbr_df['qb_rank'] = qbr_df['qb_rank'].fillna(qbr_df.qb_rank.median())
qbr_df['qb_pts'] = qbr_df['qb_pts'].fillna(qbr_df.qb_pts.median())
qbr_df['qb_plays'] = qbr_df['qb_plays'].fillna(qbr_df.qb_plays.median())
qbr_df['qb_qbr_raw'] = qbr_df['qb_qbr_raw'].fillna(qbr_df.qb_qbr_raw.median())
qbr_df['qb_qbr'] = qbr_df['qb_qbr'].fillna(qbr_df.qb_qbr.median())
# qbr_df['qb_sack'] = qbr_df['qb_sack'].fillna(qbr_df.qb_sack.median())
# qbr_df['qb_exp_sack'] = qbr_df['qb_exp_sack'].fillna(qbr_df.qb_exp_sack.median())
qbr_df['qb_pass'] = qbr_df['qb_pass'].fillna(qbr_df.qb_pass.median())
qbr_df['qb_run'] = qbr_df['qb_run'].fillna(qbr_df.qb_run.median())

# Drop unused columns
qbr_df.drop(['game_id','season','week'], axis=1, inplace=True)

qbr_df.shape

In [ ]:
print('team count:', qbr_df.posteam.nunique())

In [ ]:
qbr_df.head(2)

#### Import game rosters

In [ ]:
gr_data = nfl.import_rosters(years=seasons)

# Clean dataframe using nfl-data-api cleaning function
gr_df = nfl.clean_nfl_data(gr_data)

gr_df.shape

In [ ]:
print('team count:', gr_df.team.nunique())

In [ ]:
# Impute missing height with median by position
gr_df['height'] = gr_df.groupby(['position'])['height'].transform(lambda x: x.fillna(x.median()))
gr_df['weight'] = gr_df.groupby(['position'])['weight'].transform(lambda x: x.fillna(x.median()))

gr_df['weight'] = gr_df['weight'].astype(np.int64)

gr_df.sample(1)

#### import depth charts

#### Import snap counts

In [ ]:
sc_data = nfl.import_snap_counts(years=seasons)

# Clean dataframe using nfl-data-api cleaning function
sc_df = nfl.clean_nfl_data(sc_data)

sc_df.shape

In [ ]:
print('team count:', sc_df.team.nunique())

In [ ]:
sc_df.sample(2)

#### Import combine data

In [ ]:
co_data = nfl.import_combine_data(years=draft_years)

# Clean dataframe using nfl-data-api cleaning function
co_df = nfl.clean_nfl_data(co_data)

co_df.shape

In [ ]:
co_df['pfr_id'].fillna('NULL', inplace = True)
co_df = co_df.loc[co_df['pfr_id'] != 'NULL']

co_df = co_df.rename(columns={'pfr_id':'pfr_player_id',
                              'season':'combine',
                              'player_name':'comb_name',
                              'school':'comb_school',
                              'ht':'comb_ht',
                              'wt':'comb_wt',
                              'pos':'comb_pos',
                              'forty':'comb_forty',
                              'bench':'comb_bench',
                              'vertical':'comb_vert',
                              'broad_jump':'comb_broad',
                              'cone':'comb_cone',
                              'shuttle':'comb_shut',
                              })

co_df.drop(['draft_year','draft_team','draft_round','draft_ovr','cfb_id'], axis=1, inplace=True)

co_df.shape

In [ ]:
# Convert height to inches
co_df['comb_ht'].fillna('NULL', inplace = True)
co_df['comb_ht'] = np.where(co_df['comb_ht'] == 'NULL', '0-0', co_df['comb_ht'])
co_df['comb_ht'] = (co_df['comb_ht'].astype(str).str[0]).astype(float) * 12 + (co_df['comb_ht'].astype(str).str[2]).astype(float)

In [ ]:
co_df.sample(1)

#### Import NFL draft selections

In [ ]:
dr_data = nfl.import_draft_picks(years=draft_years)

# Clean dataframe using nfl-data-api cleaning function
dr_df = nfl.clean_nfl_data(dr_data)

dr_df.shape

In [ ]:
dr_df['team'] = np.where(dr_df['team'] == 'GNB', 'GB', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'KAN', 'KC', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'RAM', 'LA', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'STL', 'LA', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'SDG', 'LAC', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'RAI', 'LV', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'OAK', 'LV', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'LVR', 'LV', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'LAR', 'LV', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'NWE', 'NE', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'NOR', 'NO', dr_df['team'])

dr_df['team'] = np.where(dr_df['team'] == 'SFO', 'SF', dr_df['team'])
dr_df['team'] = np.where(dr_df['team'] == 'TAM', 'TB', dr_df['team'])

print('team count:', dr_df.team.nunique())

In [ ]:
# Calculate years in the NFL
dr_df['to'].fillna((dr_df['season']-1), inplace=True)

dr_df['nfl_years'] = dr_df['to'] - dr_df['season'] + 1

In [ ]:
# Conver HOF to binary
dr_df['hof'] = np.where(dr_df['hof'] == True, 1, 0)

In [ ]:
dr_df['pfr_player_id'].fillna('NULL', inplace = True)
dr_df = dr_df.loc[dr_df['pfr_player_id'] != 'NULL']

dr_df = dr_df.rename(columns={'season':'draft',
                              'team':'draft_team',
                              'pfr_player_name':'player_name',
                              'probowls':'pro_bowls'})

dr_df.drop(['cfb_player_id','gsis_id','w_av','car_av','dr_av','pass_attempts','pass_completions','pass_yards','pass_tds','pass_ints','rush_atts','rush_yards','rush_tds','receptions','rec_yards','rec_tds','def_solo_tackles','def_ints','def_sacks','to'], axis=1, inplace=True)

dr_df.shape

## Join draft data with combine data

In [ ]:
print('combine records:', co_df.shape[0])
print('draft records:', dr_df.shape[0])

In [ ]:
draft_df = pd.merge(dr_df, co_df, how='inner', left_on=['pfr_player_id'], right_on = ['pfr_player_id'])

draft_df.shape

In [ ]:
draft_df = draft_df.loc[(draft_df['category'] != 'P') & 
                        (draft_df['category'] != 'K') & 
                        (draft_df['category'] != 'LS')]

draft_df.shape

In [ ]:
# Impute missing values
draft_df['age'] = draft_df.age.fillna(draft_df.groupby('category').age.transform('median'))
draft_df['comb_ht'] = draft_df.comb_ht.fillna(draft_df.groupby('category').comb_ht.transform('median'))
draft_df['comb_wt'] = draft_df.comb_wt.fillna(draft_df.groupby('category').comb_wt.transform('median'))
draft_df['comb_forty'] = draft_df.comb_forty.fillna(draft_df.groupby('category').comb_forty.transform('median'))
draft_df['comb_bench'] = draft_df.comb_bench.fillna(draft_df.groupby('category').comb_bench.transform('median'))
draft_df['comb_vert'] = draft_df.comb_vert.fillna(draft_df.groupby('category').comb_vert.transform('median'))
draft_df['comb_broad'] = draft_df.comb_broad.fillna(draft_df.groupby('category').comb_broad.transform('median'))
draft_df['comb_cone'] = draft_df.comb_cone.fillna(draft_df.groupby('category').comb_cone.transform('median'))
draft_df['comb_shut'] = draft_df.comb_shut.fillna(draft_df.groupby('category').comb_shut.transform('median'))
draft_df['games'].fillna(0, inplace = True)

In [ ]:
# Count missing data
missing = draft_df.isnull().sum()
missing

In [ ]:
draft_df.sample(5)

In [ ]:
# Export binary classifier dataframe
draft_df.to_csv(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/nfl_post_processing_draft_data.csv', index=None, header=True)

## Join play-by-play data with supplemental data sources

In [ ]:
play_df.shape

In [ ]:
# Merge play-by-play and QBR data
# df = pd.merge(play_df, qbr_df, how='inner', left_on=['game_alt_id','posteam'], right_on = ['game_alt_id','posteam'])

# df = df.rename(columns={'game_id_x': 'game_id'})

df = play_df.copy()

df.shape
# 99226 without qbr data

In [ ]:
df.sample(1)

# _______________
## Primary play-by-play filters

In [ ]:
# Exclude columns with the following records
df = df.loc[~(df['desc'].str.startswith("END |END_|Two-Minute|Two Minute|Two minute|Two-minute|Two minute|Two-min"))]

# Exclude columns with specific key words
df = df.loc[~(df['desc'].str.contains("Captains:|CAPTAINS:|Captians:|Captains #|Captians #"))]
df = df.loc[~(df['desc'].str.contains("substitution infraction|Two-Minute Warning"))]
df = df.loc[~(df['desc'].str.contains("game has been suspended|game has resumed|Game was resumed"))]
df = df.loc[~(df['desc'].str.contains("Game delayed|game delayed|Game suspended|Game was resumed"))]
df = df.loc[~(df['desc'].str.contains("no play run|Humidity|weather delay|severe weather"))]
df = df.loc[df['play_deleted'] == 0]
df = df.loc[df['qtr'] <= 4.0]
df = df.loc[df['down'] >= 1.0]
df = df.loc[df['special_teams_play'] == 0]
df = df.loc[df['season_type'] != 'PRE']
df = df.loc[df['aborted_play'] == 0]
df = df.loc[~(df['play_type'].isin(['punt','kickoff','extra_point','field_goal','qb_kneel','qb_spike']))]
df = df.loc[~df['drive'].isnull()]

df.shape

In [ ]:
# Print a list of columns with only one value and then drop them
single_value_columns = df.loc[:,df.nunique() == 1].columns
df = df.loc[:,df.nunique() > 1]

print('Single value columns:', single_value_columns)
print('Remaining columns:', df.shape[1])

In [ ]:
# Convert field to datetime format
df['game_date']= pd.to_datetime(df['game_date'])

# Sort dataframe
df = df.sort_values(by=['posteam','game_date','play'], ascending=True)

df.shape

## Modify dataframe

In [ ]:
# Convert missing values to Null
df = df.fillna(value=np.nan)

In [ ]:
# Impute missing values
df['time_of_day'].fillna(method='ffill', inplace=True)
df['quarter_seconds_remaining'].fillna(method='bfill', inplace=True)
df['half_seconds_remaining'].fillna(method='bfill', inplace=True)
df['game_seconds_remaining'].fillna(method='bfill', inplace=True)
df['wp'].fillna(method='bfill', inplace=True)
df['def_wp'].fillna(method='bfill', inplace=True)


In [ ]:
# Remove commas from strings
df['offense_personnel'] = df['offense_personnel'].str.replace(',', '')
df['defense_personnel'] = df['defense_personnel'].str.replace(',', '')

In [ ]:
# Strip all whitespace from strings
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

# Strips all objects in dataframe
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [ ]:
# Impute missing values
df['play_type'].fillna('no_play', inplace = True)
df['play_type_nfl'].fillna('NO_PLAY', inplace = True)
# df['weekday'].fillna('NULL', inplace = True)  # included only when QBR/schedule data is include
df['sp'].fillna(0, inplace = True)
df['qtr'].fillna(0, inplace = True)
df['goal_to_go'].fillna(0, inplace = True)
df['ydstogo'].fillna(0, inplace = True) 
df['ydsnet'].fillna(0, inplace = True)
df['shotgun'].fillna(0, inplace = True)
df['no_huddle'].fillna(0, inplace = True)
df['qb_dropback'].fillna(0, inplace = True)
df['qb_kneel'].fillna(0, inplace = True)
df['qb_scramble'].fillna(0, inplace = True)
df['yards_after_catch'].fillna(0, inplace = True)
df['home_timeouts_remaining'].fillna(0, inplace = True)
df['away_timeouts_remaining'].fillna(0, inplace = True)
df['timeout'].fillna(0, inplace = True)
df['posteam_timeouts_remaining'].fillna(0, inplace = True) 
df['defteam_timeouts_remaining'].fillna(0, inplace = True)
df['total_home_score'].fillna(0, inplace = True)
df['total_away_score'].fillna(0, inplace = True)
df['posteam_score'].fillna(0, inplace = True) 
df['defteam_score'].fillna(0, inplace = True)
df['score_differential'].fillna(0, inplace = True)
df['posteam_score_post'].fillna(0, inplace = True)
df['defteam_score_post'].fillna(0, inplace = True)
df['score_differential_post'].fillna(0, inplace = True)
df['first_down_rush'].fillna(0, inplace = True)
df['first_down_pass'].fillna(0, inplace = True)
df['first_down_penalty'].fillna(0, inplace = True)
df['third_down_converted'].fillna(0, inplace = True)
df['third_down_failed'].fillna(0, inplace = True)
df['fourth_down_converted'].fillna(0, inplace = True)
df['fourth_down_failed'].fillna(0, inplace = True)
df['incomplete_pass'].fillna(0, inplace = True)
df['touchback'].fillna(0, inplace = True)
df['interception'].fillna(0, inplace = True)
df['fumble_forced'].fillna(0, inplace = True)
df['fumble_not_forced'].fillna(0, inplace = True)
df['fumble_out_of_bounds'].fillna(0, inplace = True)
df['solo_tackle'].fillna(0, inplace = True)
df['penalty'].fillna(0, inplace = True)
df['tackled_for_loss'].fillna(0, inplace = True)
df['fumble'].fillna(0, inplace = True)
df['fumble_lost'].fillna(0, inplace = True)
df['qb_hit'].fillna(0, inplace = True)
df['rush_attempt'].fillna(0, inplace = True)
df['pass_attempt'].fillna(0, inplace = True)
df['sack'].fillna(0, inplace = True)
df['safety'].fillna(0, inplace = True)
df['touchdown'].fillna(0, inplace = True)
df['pass_touchdown'].fillna(0, inplace = True)
df['rush_touchdown'].fillna(0, inplace = True)
df['return_touchdown'].fillna(0, inplace = True)
df['complete_pass'].fillna(0, inplace = True)
df['assist_tackle'].fillna(0, inplace = True)
df['lateral_reception'].fillna(0, inplace = True)
df['lateral_rush'].fillna(0, inplace = True)
df['lateral_return'].fillna(0, inplace = True)
df['lateral_recovery'].fillna(0, inplace = True)
df['passing_yards'].fillna(0, inplace = True)
df['receiving_yards'].fillna(0, inplace = True)
df['rushing_yards'].fillna(0, inplace = True)
df['lateral_receiving_yards'].fillna(0, inplace = True)
df['lateral_rushing_yards'].fillna(0, inplace = True)
df['tackle_with_assist'].fillna(0, inplace = True)
df['return_yards'].fillna(0, inplace = True)
df['replay_or_challenge'].fillna(0, inplace = True)
df['series_success'].fillna(0, inplace = True)
df['order_sequence'].fillna(0, inplace = True)
df['fixed_drive'].fillna(0, inplace = True)
df['drive_play_count'].fillna(0, inplace = True)
df['drive_first_downs'].fillna(0, inplace = True)
df['drive_inside20'].fillna(0, inplace = True)
df['drive_ended_with_score'].fillna(0, inplace = True)
df['drive_quarter_start'].fillna(0, inplace = True)
df['drive_quarter_end'].fillna(0, inplace = True)
df['drive_yards_penalized'].fillna(0, inplace = True)
df['drive_play_id_started'].fillna(0, inplace = True)
df['drive_play_id_ended'].fillna(0, inplace = True)
df['success'].fillna(0, inplace = True)
df['pass'].fillna(0, inplace = True)
df['rush'].fillna(0, inplace = True)
df['first_down'].fillna(0, inplace = True)
df['play'].fillna(0, inplace = True)
df['out_of_bounds'].fillna(0, inplace = True)
df['home_opening_kickoff'].fillna(0, inplace = True)
df['fumble_recovery_1_yards'].fillna(0, inplace = True)
df['fumble_recovery_2_yards'].fillna(0, inplace = True)
df['penalty_yards'].fillna(0, inplace = True)

In [ ]:
# Convert to integer
df['play_id'] = df['play_id'].astype(int)
df['quarter_seconds_remaining'] = df['quarter_seconds_remaining'].astype(int)
df['half_seconds_remaining'] = df['half_seconds_remaining'].astype(int)
df['game_seconds_remaining'] = df['game_seconds_remaining'].astype(int)
df['sp'] = df['sp'].astype(int)
df['qtr'] = df['qtr'].astype(int)
df['goal_to_go'] = df['goal_to_go'].astype(int)
df['ydstogo'] = df['ydstogo'].astype(int)
# df['ydsnet'] = df['ydsnet'].astype(int)
df['shotgun'] = df['shotgun'].astype(int)
df['no_huddle'] = df['no_huddle'].astype(int)
df['qb_dropback'] = df['qb_dropback'].astype(int)
df['qb_scramble'] = df['qb_scramble'].astype(int)
df['yards_after_catch'] = df['yards_after_catch'].astype(int)
df['home_timeouts_remaining'] = df['home_timeouts_remaining'].astype(int)
df['away_timeouts_remaining'] = df['away_timeouts_remaining'].astype(int)
df['timeout'] = df['timeout'].astype(int)
df['posteam_timeouts_remaining'] = df['posteam_timeouts_remaining'].astype(int)
df['defteam_timeouts_remaining'] = df['defteam_timeouts_remaining'].astype(int)
df['total_home_score'] = df['total_home_score'].astype(int)
df['total_away_score'] = df['total_away_score'].astype(int)
df['posteam_score'] = df['posteam_score'].astype(int)
df['defteam_score'] = df['defteam_score'].astype(int)
df['score_differential'] = df['score_differential'].astype(int)
df['posteam_score_post'] = df['posteam_score_post'].astype(int)
df['defteam_score_post'] = df['defteam_score_post'].astype(int)
df['score_differential_post'] = df['score_differential_post'].astype(int)
df['first_down_rush'] = df['first_down_rush'].astype(int)
df['first_down_pass'] = df['first_down_pass'].astype(int)
df['first_down_penalty'] = df['first_down_penalty'].astype(int)
df['third_down_converted'] = df['third_down_converted'].astype(int)
df['third_down_failed'] = df['third_down_failed'].astype(int)
df['fourth_down_converted'] = df['fourth_down_converted'].astype(int)
df['fourth_down_failed'] = df['fourth_down_failed'].astype(int)
df['incomplete_pass'] = df['incomplete_pass'].astype(int)
df['touchback'] = df['touchback'].astype(int)
df['interception'] = df['interception'].astype(int)
df['fumble_forced'] = df['fumble_forced'].astype(int)
df['fumble_not_forced'] = df['fumble_not_forced'].astype(int)
df['fumble_out_of_bounds'] = df['fumble_out_of_bounds'].astype(int)
df['solo_tackle'] = df['solo_tackle'].astype(int)
df['safety'] = df['safety'].astype(int)
df['penalty'] = df['penalty'].astype(int)
df['tackled_for_loss'] = df['tackled_for_loss'].astype(int)
df['fumble_lost'] = df['fumble_lost'].astype(int)
df['qb_hit'] = df['qb_hit'].astype(int)
df['rush_attempt'] = df['rush_attempt'].astype(int)
df['pass_attempt'] = df['pass_attempt'].astype(int)
df['sack'] = df['sack'].astype(int)
df['touchdown'] = df['touchdown'].astype(int)
df['rush_touchdown'] = df['rush_touchdown'].astype(int)
df['pass_touchdown'] = df['pass_touchdown'].astype(int)
df['return_touchdown'] = df['return_touchdown'].astype(int)
df['fumble'] = df['fumble'].astype(int)
df['complete_pass'] = df['complete_pass'].astype(int)
df['assist_tackle'] = df['assist_tackle'].astype(int)
df['lateral_reception'] = df['lateral_reception'].astype(int)
df['lateral_rush'] = df['lateral_rush'].astype(int)
df['lateral_return'] = df['lateral_return'].astype(int)
df['lateral_recovery'] = df['lateral_recovery'].astype(int)
df['passing_yards'] = df['passing_yards'].astype(int)
df['receiving_yards'] = df['receiving_yards'].astype(int)
df['rushing_yards'] = df['rushing_yards'].astype(int)
df['lateral_receiving_yards'] = df['lateral_receiving_yards'].astype(int)
df['lateral_rushing_yards'] = df['lateral_rushing_yards'].astype(int)
df['tackle_with_assist'] = df['tackle_with_assist'].astype(int)
df['return_yards'] = df['return_yards'].astype(int)
df['replay_or_challenge'] = df['replay_or_challenge'].astype(int)
df['series'] = df['series'].astype(int)
df['series_success'] = df['series_success'].astype(int)
df['order_sequence'] = df['order_sequence'].astype(int)
df['fixed_drive'] = df['fixed_drive'].astype(int)
df['drive_play_count'] = df['drive_play_count'].astype(int)
df['drive_first_downs'] = df['drive_first_downs'].astype(int)
df['drive_inside20'] = df['drive_inside20'].astype(int)
df['drive_ended_with_score'] = df['drive_ended_with_score'].astype(int)
df['drive_quarter_start'] = df['drive_quarter_start'].astype(int)
df['drive_quarter_end'] = df['drive_quarter_end'].astype(int)
df['drive_yards_penalized'] = df['drive_yards_penalized'].astype(int)
df['drive_play_id_started'] = df['drive_play_id_started'].astype(int)
df['drive_play_id_ended'] = df['drive_play_id_ended'].astype(int)
df['success'] = df['success'].astype(int)
df['pass'] = df['pass'].astype(int)
df['rush'] = df['rush'].astype(int)
df['first_down'] = df['first_down'].astype(int)
df['play'] = df['play'].astype(int)
df['out_of_bounds'] = df['out_of_bounds'].astype(int)
df['home_opening_kickoff'] = df['home_opening_kickoff'].astype(int)
df['fumble_recovery_1_yards'] = df['fumble_recovery_1_yards'].astype(int)
df['fumble_recovery_2_yards'] = df['fumble_recovery_2_yards'].astype(int)
df['penalty_yards'] = df['penalty_yards'].astype(int)
# df['home_rest'] = df['home_rest'].astype(int)
# df['away_rest'] = df['away_rest'].astype(int)
df.shape

In [ ]:
counts = df.play_type_nfl.value_counts(normalize=False)
counts

In [ ]:
counts = df.play_type.value_counts(normalize=False)
counts

### Impute play_type

In [ ]:
df1 = df.copy()

df1.shape

In [ ]:
# Modify description property
df1['desc'] = df1['desc'].str.strip().str.lower()

# QB Scramble
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('scrambles ')>= 0), 'qb_scramble', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'run') & (df1['desc'].str.find('scrambles ')>= 0), 'qb_scramble', df1['play_type'])

# Kickoffs
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('kicks')>= 0), 'kickoff', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('kick formation')>= 0), 'kickoff', df1['play_type'])

# Field goals
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('field goal')>= 0), 'field_goal', df1['play_type'])

# Punts
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' punts')>= 0), 'punt', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' punt is')>= 0), 'punt', df1['play_type'])

# Extra points (PATs)
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' extra point')>= 0) & (df1['desc'].str.find('penalty')>= 0), 'extra_point', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('two-point conversion')>= 0), 'two_point', df1['play_type'])

# Kneel
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' kneels')>= 0) & (df1['desc'].str.find('penalty')>= 0), 'qb_kneel', df1['play_type'])

# QB spikes
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find(' spiked')>= 0) & (df1['desc'].str.find('penalty')>= 0), 'qb_spike', df1['play_type'])

# Passes
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass incomplete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass complete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass short')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass deep')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass to')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('sacked')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass intended')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('pass intercepted')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass incomplete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass complete')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass short')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass deep')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass to')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('sacked')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass intended')>= 0), 'pass', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('pass intercepted')>= 0), 'pass', df1['play_type'])

# Runs
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('left end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('left tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('left guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('up the middle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('right guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('right tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('right end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('end around')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('left end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('left tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('left guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('up the middle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('right guard')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('right tackle')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('right end')>= 0), 'run', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'timeout') & (df1['desc'].str.find('end around')>= 0), 'run', df1['play_type'])

In [ ]:
# Penalties
df1['play_type'] = np.where(df1['desc'].str.find('false start')>= 0, 'penalty', df1['play_type'])
df1['play_type'] = np.where(df1['desc'].str.find('delay of game')>= 0, 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('penalty') >= 0) & (df1['desc'].str.find('neutral zone')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('delay of kickoff')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('too many men')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('encroachment')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('12 on-field')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('unsportsmanlike')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('interference')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('holding')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('illegal')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('offside')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('roughness')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('chop block')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('tripping')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('roughing')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('face mask')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('personal foul')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('disqualification')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('taunting')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('intentional grounding')>= 0), 'penalty', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('penalty')>= 0) & (df1['desc'].str.find('play over the down')>= 0), 'penalty', df1['play_type'])

# Replay reviews
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('replay was upheld')>= 0), 'replay_review', df1['play_type'])
df1['play_type'] = np.where((df1['play_type'] == 'no_play') & (df1['desc'].str.find('replay assistant')>= 0), 'replay_review', df1['play_type'])

# Timeouts
df1['play_type'] = np.where((df1['desc'].str.find('timeout #')>= 0) & (df1['desc'].str.find('passer_jersey_number')>= 0) &  (df1['desc'].str.find('rusher_jersey_number')>= 0), 'timeout', df1['play_type'])
df1['play_type'] = np.where((df1['desc'].str.find('timeout at')>= 0) & (df1['desc'].str.find('passer_jersey_number')>= 0) &  (df1['desc'].str.find('rusher_jersey_number')>= 0), 'timeout', df1['play_type'])

# Clock runoff (no play)
df1['play_type'] = np.where(df1['desc'].str.find('end of quarter due to 10 second clock run-off')>= 0, 'clock_runoff', df1['play_type'])

# Replay down (no play)
df1['play_type'] = np.where(df1['desc'].str.find('play over the down')>= 0, 'replay_down', df1['play_type'])
df1['play_type'] = np.where(df1['desc'].str.find('play the down')>= 0, 'replay_down', df1['play_type'])

In [ ]:
df1['play_type_nfl'] = np.where(df1['play_type'] == 'run', 'RUSH',
                                np.where(df1['play_type'] == 'penalty', 'PENALTY',
                                         np.where(df1['play_type'] == 'extra_point', 'XP_KICK',
                                                  np.where(df1['play_type'] == 'field_goal', 'FIELD_GOAL',
                                                           np.where(df1['play_type'] == 'timeout', 'TIMEOUT',
                                                                    np.where(df1['play_type'] == 'two_point', 'PAT2',
                                                                             df1['play_type_nfl']))))))

# Impute down property
df1['down'] = np.where(df1['play_type'] == 'kickoff', 0,
                       np.where(df1['play_type'] == 'extra_point', 0,
                                np.where(df1['play_type'] == 'two_point', 0,
                                         np.where(df1['play_type'] == 'injury',  df1['down'].shift(-1),
                                                  np.where(df1['play_type'] == 'replay',  df1['down'].shift(-1),
                                                           np.where(df1['play_type'] == 'timeout', df1['down'].shift(1),
                                                                    np.where(df1['play_type'] == 'penalty', df1['down'].shift(1),
                                                                             df1['down'])))))))

# Remove kickoff from drive groupings
df1['drive'] = np.where(df1['play_type'] == 'kickoff', np.nan, df1['drive'])

# Offsetting penalties
df1['offsetting_penalties'] = np.where((df1['desc'].str.find('offsetting') >= 0) & (df1['desc'].str.find('no play') >= 0), 1, 0)

# Impute missing yards_gained
df1['yards_gained'] = np.where(df1['offsetting_penalties'] == 1, 0, df1['yards_gained'])
df1['yards_gained'] = np.where(df1['play_type'] == 'timeout', 0, df1['yards_gained'])
df1['yards_gained'] = np.where((df1['play_type']=='penalty') & (df1['yards_gained']==np.nan),0,df1['yards_gained'])

## PLAY FILTERS

In [ ]:
df2 = df1.copy()

df2.shape

In [ ]:
counts = df2.play_type_nfl.value_counts(normalize=False)
counts

In [ ]:
counts = df2.play_type.value_counts(normalize=False, dropna=False)
counts

In [ ]:
df2 = df2[~df2['play_type'].isin(['punt', 'qb_kneel', 'field_goal', 'qb_scramble'])]

df2.shape

In [ ]:
counts = df2.play_type.value_counts(normalize=False, dropna=False)
counts

## FEATURE ENGINEERING

In [ ]:
# Game-play sequence
df2['play_sequence_game'] = df2.groupby(['game_id'])['play_id'].cumcount() + 1

# Game-play-drive sequence
df2['play_sequence_series'] = df2.groupby(['game_id', 'drive']).cumcount() + 1

# Code turnover on play
df2['turnover'] = np.where((df2['fumble_lost'] == 1) | (df2['interception'] == 1), 1, 0)

In [ ]:
df2['play_type_detail'] = np.where((df2['play_type']=='run') & (df2['desc'].str.find('left end')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('right end')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('end around')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('left tackle')>= 0), 'run_outside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('left guard')>= 0), 'run_inside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('middle')>= 0), 'run_inside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('right guard')>= 0), 'run_inside',
                          np.where((df2['play_type']=='run') & (df2['desc'].str.find('right tackle')>= 0), 'run_outside',

                          np.where(df2['pass_length']=='deep', 'pass_deep',
                          np.where((df2['play_type']=='pass') & (df2['air_yards']>10), 'pass_deep',
                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('pass deep')>= 0),'pass_deep',
                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('complete deep')>= 0), 'pass_deep',

                          np.where(df2['pass_length']=='short', 'pass_short',
                          np.where((df2['play_type']=='pass') & (df2['air_yards']<= 10), 'pass_short',

                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('sack')>= 0),'pass_sack',
                          np.where((df2['play_type']=='pass') & (df2['air_yards']==np.nan), 'pass',

                          np.where((df2['play_type']=='pass') & (df2['desc'].str.find('aborted')>= 0),'aborted',
                          df2['play_type'])))))))))))))))))

counts = df2.play_type_detail.value_counts(normalize=True)
counts

In [ ]:
# Impute nan values to NULL
df2['surface'] = np.where(df2['surface'] == '', np.nan, df2['surface'])

# Map stadiums to new values
stadium_map = {'Azteca Stadium': 'grass',
               'FedExField': 'grass',
               'State Farm Stadium': 'grass',
               'Mercedes-Benz Stadium': 'turf',
               'M&T Bank Stadium': 'grass',
               'Paycor Stadium': 'turf',
               'MetLife Stadium':'turf',
               'Highmark Stadium':'turf',
               'Bank of America Stadium': 'grass',
               'Soldier Field': 'grass',
               'Raymond James Stadium': 'grass',
               'Cleveland Browns Stadium': 'grass', 
               'Acrisure Stadium': 'grass',
               'AT&T Stadium': 'turf',
               'Wembley Stadium': 'grass',
               'Empower Field at Mile High': 'grass', 
               'GEHA Field at Arrowhead Stadium': 'grass', 
               'Ford Field': 'turf',
               'Tottenham Hotspur Stadium': 'turf',
               'NRG Stadium': 'turf',
               'Lucas Oil Stadium': 'turf',
               'EverBank Stadium': 'grass',
               "Levi's Stadium": 'grass',
               'Lumen Field': 'turf',
               'SoFi Stadium': 'turf', 
               'Nissan Stadium': 'turf',
               'Gillette Stadium': 'turf',
               'U.S. Bank Stadium': 'turf',
               'Lincoln Financial Field': 'grass',
               'Caesars Superdome': 'turf',
               'Allianz Arena': 'turf',
               }

# Group surface types
df2['surface'] = df2['stadium'].map(stadium_map).fillna(df2['surface'])

df2.surface.value_counts(dropna=False)

In [ ]:
# Formation properties
df2['pass_formation'] = np.where(df2['desc'].str.find('pass formation') >= 0, 1, 0)
df2['run_formation'] = np.where(df2['desc'].str.find('run formation') >= 0, 1, 0)

In [ ]:
df2.offense_formation.unique()

In [ ]:
df2['offense_formation'] = df2['offense_formation'].str.strip().str.lower()

# Impute missing offense_formation with pass formation and run formation
df2['offense_formation'] = np.where((df2['offense_formation'].isnull()) & (df2['shotgun'] == 1),'shotgun', 
                                    np.where((df2['offense_formation'].isnull()) & (df2['pass_formation'] == 1),'empty',
                                             np.where((df2['offense_formation'].isnull()) & (df2['run_formation'] == 1),'jumbo',
                                                      np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str[0:4] == '6 OL'),'jumbo',
                                                               np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str.find('3 TE') >0),'jumbo',
                                                                        np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str.find('0 WR') >0),'jumbo',
                                                                                 np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str.find('3 WR') >0), 'shotgun',
                                                                                          np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str[0:4] == '2 RB'), 'shotgun',
                                                                                                   np.where((df2['offense_formation'].isnull()) & (df2['offense_personnel'].str[0:4] == '1 RB'), 'singleback',
                                                                                                            np.where(df2['offense_formation'].isnull(), 'shotgun',
                                                                                                                     df2['offense_formation']))))))))))


In [ ]:
df2.offense_formation.value_counts(dropna=False)

In [ ]:
# Offsetting penalties
df2['offsetting_penalties'] = np.where((df2['desc'].str.find('offsetting') >= 0) & (df2['desc'].str.find('no play') >= 0), 1, 0)

# Impute missing yards_gained
df2['yards_gained'] = np.where(df2['offsetting_penalties'] == 1, 0, df2['yards_gained'])

In [ ]:
# Code yardline zones
df2['dtg_99to95'] = np.where(df2['yardline_100'] >= 95, 1, 0)
df2['dtg_94to90'] = np.where((df2['yardline_100'] <= 94) & (df2['yardline_100'] >= 90), 1, 0)
df2['dtg_40to31'] = np.where((df2['yardline_100'] <= 40) & (df2['yardline_100'] >= 31), 1, 0)
df2['dtg_30to21'] = np.where((df2['yardline_100'] <= 30) & (df2['yardline_100'] >= 21), 1, 0)
df2['dtg_20to11'] = np.where((df2['yardline_100'] <= 20) & (df2['yardline_100'] >= 11), 1, 0)
df2['dtg_10to06'] = np.where((df2['yardline_100'] <= 10) & (df2['yardline_100'] >= 6), 1, 0)
df2['dtg_05to01'] = np.where(df2['yardline_100'] <= 5, 1, 0)

In [ ]:
# Code big gains on previous run plays (>= 15 yards)
df2['big_play_pass'] = np.where((df2['play_type']=='pass') & (df2['yards_gained']>=15) & (df2['turnover']==0), 1, 0)

df2['prev1_big_play_pass'] = df2['big_play_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_big_play_pass'] = df2['big_play_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_big_play_pass'] = df2['big_play_pass'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_big_play_pass_pcnt'] = (df2.groupby(['game_id', 'drive'])['big_play_pass'].transform(lambda x: x.rolling(window=50, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

In [ ]:
# Code big gains on previous run plays (>= 7 yards)
df2['big_play_run'] = np.where((df2['play_type']=='run') & (df2['yards_gained']>=7) & (df2['turnover']==0), 1, 0)

df2['prev1_big_play_run'] = df2['big_play_run'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_big_play_run'] = df2['big_play_run'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_big_play_run'] = df2['big_play_run'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_big_play_run_pcnt'] = (df2.groupby(['game_id', 'drive'])['big_play_run'].transform(lambda x: x.rolling(window=50, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

In [ ]:
# Code negative run on previous plays
df2['negative_pass'] = np.where((df2['play_type']=='pass') & (df2['yards_gained']<0) & (df2['turnover']==0), 1, 0)

df2['prev1_negative_pass'] = df2['negative_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_negative_pass'] = df2['negative_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_negative_pass'] = df2['negative_pass'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

In [ ]:
# Code negative run on previous plays
df2['negative_run'] = np.where((df2['play_type']=='run') & (df2['yards_gained'] < 0) & (df2['turnover']==0), 1, 0)

df2['prev1_negative_run'] = df2['negative_run'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_negative_run'] = df2['negative_run'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_negative_run'] = df2['negative_run'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

In [ ]:
# Code the play_type on previous plays
df2['play_type_prev1'] = df2['play_type'].shift(1).where(df2['drive'].shift(1) == df2['drive'], '')
df2['play_type_prev2'] = df2['play_type'].shift(2).where(df2['drive'].shift(2) == df2['drive'], '')
df2['play_type_prev3'] = df2['play_type'].shift(3).where(df2['drive'].shift(3) == df2['drive'], '')

# Code binary penalty indicator on previous plays
df2['penalty_team_prev1'] = df2['penalty_team'].shift(1).where(df2['drive'].shift(1) == df2['drive'], '')
df2['penalty_team_prev2'] = df2['penalty_team'].shift(2).where(df2['drive'].shift(2) == df2['drive'], '')
df2['penalty_team_prev3'] = df2['penalty_team'].shift(3).where(df2['drive'].shift(3) == df2['drive'], '')

##########
df2['prev1_play_off_penalty'] = np.where((df2['play_type_prev1'] == 'penalty') & (df2['penalty_team_prev1'] == df2['posteam']), 1, 0)
df2['prev1_play_def_penalty'] = np.where((df2['play_type_prev1'] == 'penalty') & (df2['penalty_team_prev1'] == df2['defteam']), 1, 0)

##########
df2['prev2_play_off_penalty'] = np.where((df2['play_type_prev2'] == 'penalty') & (df2['penalty_team_prev2'] == df2['posteam']), 1, 0)
df2['prev2_play_def_penalty'] = np.where((df2['play_type_prev2'] == 'penalty') & (df2['penalty_team_prev2'] == df2['defteam']), 1, 0)

##########
df2['prev3_play_off_penalty'] = np.where((df2['play_type_prev3'] == 'penalty') & (df2['penalty_team_prev3'] == df2['posteam']), 1, 0)
df2['prev3_play_def_penalty'] = np.where((df2['play_type_prev3'] == 'penalty') & (df2['penalty_team_prev3'] == df2['defteam']), 1, 0)

#########
df2.drop(['play_type_prev1','play_type_prev2','play_type_prev3','penalty_team_prev1','penalty_team_prev2','penalty_team_prev3'], axis=1, inplace=True)

In [ ]:
# Code the play_type_detail on previous plays
df2['play_type_detail_prev1'] = df2['play_type_detail'].shift(1).where(df2['drive'].shift(1) == df2['drive'], '')
df2['play_type_detail_prev2'] = df2['play_type_detail'].shift(2).where(df2['drive'].shift(2) == df2['drive'], '')
df2['play_type_detail_prev3'] = df2['play_type_detail'].shift(3).where(df2['drive'].shift(3) == df2['drive'], '')

df2['prev1_play_run_outside'] = np.where(df2['play_type_detail_prev1'] == 'run_outside', 1, 0)
df2['prev1_play_run_inside'] = np.where(df2['play_type_detail_prev1'] == 'run_inside', 1, 0)
df2['prev1_play_pass_deep'] = np.where(df2['play_type_detail_prev1'] == 'pass_deep', 1, 0)
df2['prev1_play_pass_short'] = np.where(df2['play_type_detail_prev1'] == 'pass_short', 1, 0)

df2['prev2_play_run_outside'] = np.where(df2['play_type_detail_prev2'] == 'run_outside', 1, 0)
df2['prev2_play_run_inside'] = np.where(df2['play_type_detail_prev2'] == 'run_inside', 1, 0)
df2['prev2_play_pass_deep'] = np.where(df2['play_type_detail_prev2'] == 'pass_deep', 1, 0)
df2['prev2_play_pass_short'] = np.where(df2['play_type_detail_prev2'] == 'pass_short', 1, 0)

df2['prev3_play_run_outside'] = np.where(df2['play_type_detail_prev3'] == 'run_outside', 1, 0)
df2['prev3_play_run_inside'] = np.where(df2['play_type_detail_prev3'] == 'run_inside', 1, 0)
df2['prev3_play_pass_deep'] = np.where(df2['play_type_detail_prev3'] == 'pass_deep', 1, 0)
df2['prev3_play_pass_short'] = np.where(df2['play_type_detail_prev3'] == 'pass_short', 1, 0)

df2.drop(['play_type_detail_prev1','play_type_detail_prev2','play_type_detail_prev3'], axis=1, inplace=True)

In [ ]:
# Code incomplete passes on previous plays
df2['prev1_incomplete_pass'] = df2['incomplete_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_incomplete_pass'] = df2['incomplete_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_incomplete_pass'] = df2['incomplete_pass'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_incomplete_pass'].fillna(0, inplace = True)
df2['prev2_incomplete_pass'].fillna(0, inplace = True)
df2['prev3_incomplete_pass'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_incomplete_pass_pcnt'] = (df2.groupby(['game_id', 'drive'])['incomplete_pass'].transform(lambda x: x.rolling(window=50, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

In [ ]:
# Code yards on previous plays
df2['prev1_yards_gained'] = df2['yards_gained'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_yards_gained'] = df2['yards_gained'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_yards_gained'] = df2['yards_gained'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_yards_gained'].fillna(0, inplace = True)
df2['prev2_yards_gained'].fillna(0, inplace = True)
df2['prev3_yards_gained'].fillna(0, inplace = True)

In [ ]:
# Code win probability of previous plays
df2['prev1_wpa'] = df2['wpa'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_wpa'] = df2['wpa'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_wpa'] = df2['wpa'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_wpa'].fillna(0, inplace = True)
df2['prev2_wpa'].fillna(0, inplace = True)
df2['prev3_wpa'].fillna(0, inplace = True)

In [ ]:
# Code shotgun formation of previous plays
df2['prev1_shotgun'] = df2['shotgun'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_shotgun'] = df2['shotgun'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_shotgun'] = df2['shotgun'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_shotgun'].fillna(0, inplace = True)
df2['prev2_shotgun'].fillna(0, inplace = True)
df2['prev3_shotgun'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_shotgun_pcnt'] = (df2.groupby(['game_id', 'drive'])['shotgun'].transform(lambda x: x.rolling(window=50, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

In [ ]:
# Code qb_hit on previous plays
df2['prev1_qb_hit'] = df2['qb_hit'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_qb_hit'] = df2['qb_hit'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_qb_hit'] = df2['qb_hit'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_qb_hit'].fillna(0, inplace = True)
df2['prev2_qb_hit'].fillna(0, inplace = True)
df2['prev3_qb_hit'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_qb_hit_pcnt'] = (df2.groupby(['game_id', 'drive'])['qb_hit'].transform(lambda x: x.rolling(window=50, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

In [ ]:
# Code no huddle of previous plays
df2['prev1_no_huddle'] = df2['no_huddle'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_no_huddle'] = df2['no_huddle'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_no_huddle'] = df2['no_huddle'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_no_huddle'].fillna(0, inplace = True)
df2['prev2_no_huddle'].fillna(0, inplace = True)
df2['prev3_no_huddle'].fillna(0, inplace = True)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_no_huddle_pcnt'] = (df2.groupby(['game_id', 'drive'])['no_huddle'].transform(lambda x: x.rolling(window=50, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

In [ ]:
# Code first down achieved on a previous play
df2['prev1_first_down_pass'] = df2['first_down_pass'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_first_down_pass'] = df2['first_down_pass'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_first_down_pass'] = df2['first_down_pass'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_first_down_pass'].fillna(0, inplace = True)
df2['prev2_first_down_pass'].fillna(0, inplace = True)
df2['prev3_first_down_pass'].fillna(0, inplace = True)

In [ ]:
# Code first down achieved on a previous play
df2['prev1_first_down_run'] = df2['first_down_rush'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_first_down_run'] = df2['first_down_rush'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_first_down_run'] = df2['first_down_rush'].shift(3).where(df2['drive'].shift(2) == df2['drive'], 0)

df2['prev1_first_down_run'].fillna(0, inplace = True)
df2['prev2_first_down_run'].fillna(0, inplace = True)
df2['prev3_first_down_run'].fillna(0, inplace = True)

In [ ]:
# Code play efficiency (1st down >= 4, 2nd down half the distance, 3rd and 4th down = first down)
df2['effct_play'] = np.where((df2['down']==1) & (df2['yards_gained'] >= 4) & (df2['turnover'] == 0), 1,
                    np.where((df2['down']==2) & (df2['yards_gained'] >= (df2['ydstogo']/2)) & (df2['turnover'] == 0), 1,
                    np.where((df2['down']==3) & (df2['yards_gained'] >= df2['ydstogo']) & (df2['turnover'] == 0), 1,
                    np.where((df2['down']==4) & (df2['yards_gained'] >= df2['ydstogo']) & (df2['turnover'] == 0), 1, 0))))

df2['prev1_effct_play'] = df2['effct_play'].shift(1).where(df2['drive'].shift(1) == df2['drive'], 0)
df2['prev2_effct_play'] = df2['effct_play'].shift(2).where(df2['drive'].shift(2) == df2['drive'], 0)
df2['prev3_effct_play'] = df2['effct_play'].shift(3).where(df2['drive'].shift(3) == df2['drive'], 0)

# Calculate the percent of play classification within prior plays of current drive
df2['drive_effct_play_pcnt'] = (df2.groupby(['game_id', 'drive'])['effct_play'].transform(lambda x: x.rolling(window=50, min_periods=1, closed='left').sum())/df2['play_sequence_series'] - 1).fillna(0)

In [ ]:
# Code downs_remaining
df2['remaining_downs'] = np.where(df2['down'] == 1, 3, 
                         np.where(df2['down'] == 2, 2,
                         np.where(df2['down'] == 3, 1,
                         np.where(df2['down'] == 4, 1, 1))))

# Code yards_per_down remaining
df2['remaining_yards_per_down'] = df2['ydstogo'] / df2['remaining_downs']

In [ ]:
df2['game_half'] = np.where(df2['game_half'] == 'Half1', 1, np.where(df2['game_half'] == 'Half2', 2, 3))

In [ ]:
df2.drive_start_transition.unique()

In [ ]:
df2['drive_start_transition'] = df2['drive_start_transition'].str.strip().str.upper()

# Modify drive start category
df2['drive_start'] = np.where(df2['drive_start_transition'] == 'INTERCEPTION', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'FUMBLE', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'MUFFED_PUNT', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'ONSIDE_KICK', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'BLOCKED_FG', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'BLOCKED_PUNT', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'BLOCKED_FG_DOWNS', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'BLOCKED_FG,_DOWNS', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'BLOCKED_PUNT_DOWNS', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'MUFFED_KICKOFF', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'OWN_KICKOFF', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'DOWNS', 'sudden_change',
                     np.where(df2['drive_start_transition'] == 'MISSED_FG', 'transfer_poss',
                     np.where(df2['drive_start_transition'] == 'KICKOFF', 'transfer_poss',
                     np.where(df2['drive_start_transition'] == 'PUNT', 'transfer_poss',
                     df2['drive_start_transition'])))))))))))))))

df2['drive_start'] = df2['drive_start'].str.strip().str.lower()

df2['drive_start'].fillna('NULL', inplace=True)

df2.drive_start.unique()

In [ ]:
df2['two_min_warning'] = np.where(df2['half_seconds_remaining'] <= 120, 1, 0)

In [ ]:
df2['ep_sec_ratio'] = (df2['ep'] / (df2['half_seconds_remaining'] + 1 ))

In [ ]:
df2['posteam_season'] = df2['posteam'] + '_' + df2['season'].astype(str)

In [ ]:
# Duplicate dataframe
df3 = df2.copy()

df3.shape

In [ ]:
# Rename target classes
df3['play_type_detail'] = np.where(df3['play_type_detail'] == 'pass_short', 'short',
                                   np.where(df3['play_type_detail'] == 'pass_deep', 'deep',
                                            np.where(df3['play_type_detail'] == 'run_inside', 'inside',
                                                     np.where(df3['play_type_detail'] == 'run_outside', 'outside',
                                                              df3['play_type_detail']))))

## Run ratio feature

In [ ]:
# Calculate average run ratio per game
rro_df = df3.groupby(['posteam','game_id','qtr','down'])['rush_attempt'].apply(lambda x : x.astype(float).mean()).reset_index()

# Sort dataframe
rro_df = rro_df.sort_values(by=['posteam','qtr','down'], ascending=True)

# Calculate rolling average per quarter
rro_df['run_ratio_off_priors'] = rro_df.groupby(['posteam','qtr','down'])['rush_attempt'].transform(lambda x: x.rolling(window=3,
                                                                                                                        min_periods=1,
                                                                                                                        closed='left',
                                                                                                                        center=False).mean())

rro_df = rro_df.sort_values(by=['posteam','qtr','down'], ascending=True)

rro_df.drop(['rush_attempt'], axis=1, inplace=True)

# Impute missing values by the league average per qtr/down
rro_df['run_ratio_off_priors'] = rro_df['run_ratio_off_priors'].fillna(rro_df.groupby(['qtr','down'])['run_ratio_off_priors'].transform('mean'))

# Merge msa_df with df
df3 = df3.merge(rro_df, how='left', left_on=['posteam','game_id','qtr','down'], right_on=['posteam','game_id','qtr','down'])

In [ ]:
# Calculate average run ratio per game
rrd_df = df3.groupby(['defteam','game_id','qtr','down'])['rush_attempt'].apply(lambda x : x.astype(float).mean()).reset_index()

# Sort dataframe
rrd_df = rrd_df.sort_values(by=['defteam','qtr','down'], ascending=True)

# Calculate rolling average per quarter
rrd_df['run_ratio_def_priors'] = rrd_df.groupby(['defteam','qtr','down'])['rush_attempt'].transform(lambda x: x.rolling(window=3,
                                                                                                                        min_periods=1,
                                                                                                                        closed='left',
                                                                                                                        center=False).mean())

rrd_df = rrd_df.sort_values(by=['defteam','qtr','down'], ascending=True)

rrd_df.drop(['rush_attempt'], axis=1, inplace=True)

# Impute missing values by the league average per qtr/down
rrd_df['run_ratio_def_priors'] = rrd_df['run_ratio_def_priors'].fillna(rrd_df.groupby(['qtr','down'])['run_ratio_def_priors'].transform('mean'))

# Merge msa_df with df
df3 = df3.merge(rrd_df, how='left', left_on=['defteam','game_id','qtr','down'], right_on=['defteam','game_id','qtr','down'])

## Future features

## Play prediction filters

In [ ]:
# Filters used to eliminate non-typical offensive plays / plays with alterior incentives (preseason) 
df3 = df3.loc[df3['season_type'] != 'PRE']
df3 = df3.loc[(df3['play_type'] == 'pass') | (df3['play_type'] == 'run')]
df3 = df3.loc[df3['play_type_detail'] != 'pass_sack']
df3 = df3.loc[df3['qb_scramble'] == 0]
df3 = df3.loc[df3['sack'] == 0]
df3 = df3.loc[df3['drive_start'] != 'NULL']

df3.shape

In [ ]:
# Print a list of columns with only one value and then drop them
single_value_columns = df3.loc[:,df3.nunique() == 1].columns
df3 = df3.loc[:,df3.nunique() > 1]

print('Single value columns:', single_value_columns)
print('Remaining columns:', df3.shape[1])

In [ ]:
# Convert properties into integer
df3['yardline_100'] = df3['yardline_100'].astype(int)
df3['down'] = df3['down'].astype(int)
df3['drive'] = df3['drive'].astype(int)

In [ ]:
counts = df3.drive_start.value_counts(normalize=False)
counts

In [ ]:
counts = df3.play_type_detail.value_counts(normalize=False)
counts

## DROP COLUMNS

In [ ]:
# Drop unused features
df3.drop(['old_game_id','game_alt_id','season_type','side_of_field','game_date','drive','time','yrdln','ydsnet','desc','qb_dropback','air_yards','yards_after_catch','timeout','timeout_team','td_team','td_player_name','td_player_id','total_home_score','total_away_score','posteam_score_post','defteam_score_post','score_differential_post','opp_fg_prob','opp_safety_prob','opp_td_prob','total_home_epa','total_away_epa','total_home_rush_epa','total_away_rush_epa','total_home_pass_epa','total_away_pass_epa','air_epa','yac_epa','comp_air_epa','comp_yac_epa','total_home_comp_air_epa','total_away_comp_air_epa','total_home_comp_yac_epa','total_away_comp_yac_epa','total_home_raw_air_epa','total_away_raw_air_epa','total_home_raw_yac_epa','total_away_raw_yac_epa','def_wp','home_wp','away_wp','wpa','vegas_wpa','vegas_home_wpa','home_wp_post','away_wp_post','vegas_wp','vegas_home_wp','total_home_rush_wpa','total_away_rush_wpa','total_home_pass_wpa','total_away_pass_wpa','air_wpa','yac_wpa','comp_air_wpa','comp_yac_wpa','total_home_comp_air_wpa','total_away_comp_air_wpa','total_home_comp_yac_wpa','total_away_comp_yac_wpa','total_home_raw_air_wpa','total_away_raw_air_wpa','total_home_raw_yac_wpa','total_away_raw_yac_wpa','first_down_rush','first_down_pass','first_down_penalty','third_down_converted','third_down_failed','fourth_down_converted','fourth_down_failed','incomplete_pass','touchback','interception','fumble_forced','fumble_not_forced','fumble_out_of_bounds','solo_tackle','safety','penalty','tackled_for_loss','fumble_lost','qb_hit','touchdown','pass_touchdown','rush_touchdown','return_touchdown','fumble','complete_pass','assist_tackle','lateral_reception','lateral_rush','lateral_return','lateral_recovery','passer_player_id','passer_player_name','passing_yards','receiver_player_id','receiver_player_name','receiving_yards','rusher_player_id','rusher_player_name','rushing_yards','lateral_receiver_player_id','lateral_receiver_player_name','lateral_receiving_yards','lateral_rusher_player_id','lateral_rusher_player_name','lateral_rushing_yards','interception_player_id','interception_player_name','lateral_interception_player_id','lateral_interception_player_name','tackle_for_loss_1_player_id','tackle_for_loss_1_player_name','qb_hit_1_player_id','qb_hit_1_player_name','forced_fumble_player_1_team','forced_fumble_player_1_player_id','forced_fumble_player_1_player_name','forced_fumble_player_2_team','forced_fumble_player_2_player_id','forced_fumble_player_2_player_name','solo_tackle_1_team','solo_tackle_2_team','solo_tackle_1_player_id','solo_tackle_2_player_id','solo_tackle_1_player_name','solo_tackle_2_player_name','assist_tackle_1_player_id','assist_tackle_1_player_name','assist_tackle_1_team','assist_tackle_2_player_id','assist_tackle_2_player_name','assist_tackle_2_team','tackle_with_assist','tackle_with_assist_1_player_id','tackle_with_assist_1_player_name','tackle_with_assist_1_team','pass_defense_1_player_id','pass_defense_1_player_name','pass_defense_2_player_id','pass_defense_2_player_name','fumbled_1_team','fumbled_1_player_id','fumbled_1_player_name','fumbled_2_player_id','fumbled_2_player_name','fumbled_2_team','fumble_recovery_1_team','fumble_recovery_1_yards','fumble_recovery_1_player_id','fumble_recovery_1_player_name','fumble_recovery_2_team','fumble_recovery_2_yards','fumble_recovery_2_player_id','fumble_recovery_2_player_name','return_team','return_yards','penalty_team','penalty_player_id','penalty_player_name','penalty_yards','replay_or_challenge','replay_or_challenge_result','penalty_type','safety_player_name','safety_player_id','series','series_success','series_result','start_time','time_of_day','stadium','weather','nfl_api_id','play_clock','play_type_nfl','end_clock_time','end_yard_line','fixed_drive','fixed_drive_result','drive_real_start_time','drive_time_of_possession','drive_first_downs','drive_inside20','drive_ended_with_score','drive_quarter_start','drive_quarter_end','drive_yards_penalized','drive_end_transition','drive_start_transition','drive_game_clock_start','drive_game_clock_end','drive_start_yard_line','drive_end_yard_line','drive_play_id_started','drive_play_id_ended','away_score','home_score','location','result','total','home_coach','away_coach','stadium_id','game_stadium','success','passer','passer_jersey_number','rusher','rusher_jersey_number','receiver','receiver_jersey_number','pass','rush','first_down','passer_id','rusher_id','receiver_id','name','jersey_number','id','fantasy_player_name','fantasy_player_id','fantasy','fantasy_id','out_of_bounds','home_opening_kickoff','qb_epa','xyac_epa','xyac_mean_yardage','xyac_median_yardage','xyac_success','xyac_fd','xpass','pass_oe','cp','cpoe','pass_length','pass_location','run_location','run_gap','pass_attempt','game_id','home_team','away_team','sp','yards_gained','home_timeouts_remaining','away_timeouts_remaining','order_sequence','play_id','posteam_type','epa','turnover','temp','wind','roof','effct_play','negative_run','negative_pass','big_play_run','big_play_pass','offsetting_penalties','season','posteam_season','rush_attempt','nflverse_game_id','surface','possession_team','players_on_play','offense_players','defense_players'], axis=1, inplace=True)
# 'possession_team','offense_formation','offense_personnel', 'defenders_in_box', 'defense_personnel', 'number_of_pass_rushers', 'players_on_play',  'n_offense','n_defense','gameday','weekday','gametime','home_rest', 'away_rest'

df3.shape

In [ ]:
# list of columns names with null values
null_columns = df3.columns[df3.isnull().any()]
df3.dropna(axis=1, inplace=True)
print('Dropping the following columns with missing values:', null_columns)
print('Remaining column count:', df3.shape[1])

## VIF exclusion criteria

## Create binary classification dataset

In [ ]:
# Print list of columns
list(df3.columns)

In [ ]:
df6 = df3.copy()

# Drop column not used in classifier
df6.drop(['play_type_detail'], axis=1, inplace=True)

# Export binary classifier dataframe
df6.to_csv(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/nfl_post_processing_run_pass_classification_data.csv', index=None, header=True)

## Create multi-classification dataset

In [ ]:
df7 = df3.copy()

# Drop column not used in classifier
df7.drop(['play_type'], axis=1, inplace=True)
df7.shape

In [ ]:
# Exclude sacks from dataset
df7 = df7.loc[df7['play_type_detail'] != 'pass_sack']
df7.shape

In [ ]:
# Exclude passes with no depth designation
df7 = df7.loc[df7['play_type_detail'] != 'pass']
df7.shape

In [ ]:
# Exclude runs with no direction/landmark designation
df7 = df7.loc[df7['play_type_detail'] != 'run']
df7.shape

In [ ]:
# Rename target
df7['play_type'] = df7['play_type_detail']

# Drop duplicate target
df7.drop(['play_type_detail'], axis=1, inplace=True)

# Check class frequency
counts = df7.play_type.value_counts(normalize=True)
counts

In [ ]:
# Export binary classifier dataframe
df7.to_csv(r'/Users/ttas2/Documents/Python/nfl-machine-learning-models/output_files/nfl_post_processing_multiclass_play_classification_data.csv', index=None, header=True)